# Running *species*

This tutorial shows what a typical workflow with *species* looks like. The photometric fluxes of the brown dwarf companion PZ Tel B is compared with a spectrum from the AMES-Cond atmospheric model.

## Initiating *species*

We start by importing the *species* toolkit.

In [1]:
import species

Next, the configuration file (*species_config.ini*) is read from the working folder with `SpeciesInit`. Since the file does not exist, it is created with default values for the path of the HDF5 database and the location where the external data is stored. The database and data folder are also created since this is the first time that we run *species* with this configuration file. 

In [2]:
species.SpeciesInit('./')

Initiating species v0.1.0... [DONE]
Creating species_config.ini... [DONE]
Creating species_database.hdf5... [DONE]
Creating data folder... [DONE]


## Adding model spectra

A `Database` object is now created which can be used for importing of data into the database.

In [3]:
database = species.Database()

The spectra of the AMES-Cond model are downloaded and added to the database. This requires sufficient disk storage in the *data_folder* that is set in the configuration file.

While the full AMES-Cond grid is downloaded, a restiction on the wavelength and T<sub>eff</sub> range can be used when the data is imported. Also the spectral resolution can be set to downsample the high-resolution of the original spectra spectra.

In [ ]:
database.add_model(model='ames-cond',
                   wavel_range=(0.1, 6.),
                   teff_range=(2800., 3000.),
                   spec_res=1000.)

## Adding companion data

The `data.companions` module contains a [dictionary with apparent magnitudes](https://github.com/tomasstolker/species/blob/master/species/data/companions.py) of directly imaged planets and brown dwarfs. Let's have a look which data are available.

In [ ]:
database.list_companions()

The distance and magnitudes of PZ Tel B are added to the database. This will automatically download the filter transmission profiles and a flux calibrated spectrum of Vega, which are used for the flux calibration.

In [ ]:
database.add_companion(name='PZ Tel B')

Alternatively, the `add_object` method of `Database` can be used for manually adding magnitudes and spectra of an individual object. Before coninuing, let's check the content of the database.

In [ ]:
database.list_content()

## Reading model spectra

Model spectra are read from the database by creating an object of `ReadModel`. The model name and optionally a wavelength range should be provided. The wavelength range should fit within the range that was specified in `add_model`.

In [ ]:
readmodel = species.ReadModel(model='ames-cond',
                              wavel_range=(0.1, 6.))

Before extracting a spectrum, let's check which parameters are required for the AMES-Cond models.

In [ ]:
readmodel.get_parameters()

And also the grid boundaries that are stored in the database.

In [ ]:
readmodel.get_bounds()

The parameters are provided in a dictionary. Make sure to chose values within the grid boundaries. The radius (R<sub>Jup</sub>) and distance (pc) will scale the emitted spectrum to the observer. Without these values, the spectrum is provided at the outermost layer of the atmosphere.

In [ ]:
model_param = {'teff': 2900., 'logg': 4.5, 'radius': 2.2, 'distance': 47.13}

The grid of spectra is linearly interpolated at a spectral resolution of 100 and stored in a `ModelBox`.

In [ ]:
modelbox = readmodel.get_model(model_param=model_param,
                               spec_res=100.)

## Reading companion data

The photometry data of PZ Tel B are also read from the database and stored in an `ObjectBox`.

In [ ]:
objectbox = database.get_object(object_name='PZ Tel B')

## Creating flux residuals

The `get_residuals` function is used to calculate the difference between the observed fluxes and the synthetic photometry of the model spectrum.

In [ ]:
residualsbox = species.get_residuals(datatype='model',
                                     spectrum='ames-cond',
                                     parameters=model_param,
                                     filters=objectbox.filters,
                                     objectbox=objectbox,
                                     inc_phot=True,
                                     inc_spec=False)

## Opening Box objects

The `open_box` function can be used to view the content of a `Box` object. For example, the `ModelBox` contains several attributes, including the wavelength points and flux densities.

In [ ]:
modelbox.open_box()

Similarly, an `ObjectBox` contains a dictionary with the apparent magnitudes and a dictionary with the flux densities.

In [ ]:
objectbox.open_box()

The attributes in the `Box` objects can be extracted for further analyis or manual plots, for example:

In [ ]:
modelbox.wavelength

## Plotting spectra, fluxes and filters

The `Box` objects can provided as list to `plot_spectrum`, which will be interpreted accordingly. Optionally, the filter names can be included for plotting the transmission profiles and the `ResidualsBox` can be included for showing the residuals. The `ObjectBox` requires a color for the photometry data and a spectrum with the latter set to `None` in this case.

In [ ]:
species.plot_spectrum(boxes=[modelbox, objectbox],
                      filters=objectbox.filters,
                      residuals=residualbox,
                      colors=('black', ('tomato', None)),
                      offset=(-0.08, -0.06),
                      xlim=(0.2, 5.5),
                      ylim=(0., 4.8e-14),
                      legend='upper right',
                      output='spectrum.png')

The plot is stored in the PNG image format. Let's have a look at the result!

In [ ]:
from IPython.display import Image
Image('spectrum.png') 